In [13]:
from sudoku import Sudoku
import pandas as pd
import subprocess

In [14]:
#get main cnf file
path = 'sudoku9.cnf'
cnf =""
f = open(path, 'r')
for line in f:
    cnf+= line
f.close()

In [15]:
# Global Constants
MIN_VARS = 1
MAX_VARS = 25
INSTANCES_PER_VAR = 1
SUDOKU_DEGREE = 9
VAR_COUNT = int(str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE))
SOLVER_NAME = "minisat"

In [16]:
frames = [] # will store dataframes for each variable

In [17]:
frames = []
for var in range(MIN_VARS, MAX_VARS):
    boards = []
    frame = pd.DataFrame(columns=['board', 'solution'])
    for y in range(INSTANCES_PER_VAR):
        board = Sudoku(SUDOKU_DEGREE)
        board.randomize_board(var)
        boards.append(board)
    cnfs = [board.cnf_output() for board in boards]
    frame['board'] = boards
    frame['cnf'] = cnfs
    frame['num_vars'] = var
    frames.append(frame)
frames = pd.DataFrame(pd.concat(frames))
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7f68f1c89f28>,NaN,026 0\n,1
0,<sudoku.Sudoku object at 0x7f68f1c14e80>,NaN,269 0\n382 0\n,2
0,<sudoku.Sudoku object at 0x7f68f1c14cf8>,NaN,312 0\n385 0\n757 0\n,3
0,<sudoku.Sudoku object at 0x7f68f1c14cc0>,NaN,105 0\n285 0\n362 0\n404 0\n,4
0,<sudoku.Sudoku object at 0x7f68f1c14978>,NaN,057 0\n158 0\n648 0\n702 0\n724 0\n,5


In [18]:
# This function takes in a string, adds a header suitable to DIMAACS and writes it to the file
def write_cnf_file(fname, string):
    
    # write it initially so file exists first
    with open(fname, "w") as f:
        f.write(string)
    
    # read the # of clauses in the file
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
        clause_count = i + 1
    
    # writes the file back with the suitable header
    with open(fname, "w") as f:
        f.write("p cnf " + str(VAR_COUNT) + " " + str(clause_count) + "\n")
        f.write(string)
        
    return

In [19]:
# This function solves a DIMACS formatted SAT problem using the solver

def solve_using_bash(input_f, output_f):
    
    # the bash command we feed to the terminal
    bashCommand = SOLVER_NAME + " " + input_f + " " + output_f
    
    # where the magic happens
    process = subprocess.Popen(bashCommand.split(), stdout = subprocess.PIPE)
    
    # print any errors to stdout
    output, error = process.communicate()
    
    return

In [20]:
def get_solution_from_cnf(cnf_string):
    #do something here
    cnf_string = cnf + cnf_string #append the whole cnf
    
     # Our temporary input/output files
    input_file = "inputtemp.cnf"
    output_file = "outputtemp.txt"
    
    # write the string as a DIMACS format file
    write_cnf_file(input_file, cnf_string)
    
    # Solve using sat solver through bash command
    solve_using_bash(input_file, output_file)
    
    # open the output and send the contents 
    with open (output_file) as f:
        return f.read()

In [21]:
%%time
frames['solution'] = frames['cnf'].apply(get_solution_from_cnf)

CPU times: user 76 ms, sys: 80 ms, total: 156 ms
Wall time: 6.42 s


In [22]:
frames.head(5)


,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7f68f1c89f28>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,026 0\n,1
0,<sudoku.Sudoku object at 0x7f68f1c14e80>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,269 0\n382 0\n,2
0,<sudoku.Sudoku object at 0x7f68f1c14cf8>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,312 0\n385 0\n757 0\n,3
0,<sudoku.Sudoku object at 0x7f68f1c14cc0>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,105 0\n285 0\n362 0\n404 0\n,4
0,<sudoku.Sudoku object at 0x7f68f1c14978>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,057 0\n158 0\n648 0\n702 0\n724 0\n,5
